# **AI Web Application generator from a Topic and its Target Audience**

 **Author:** Syed Ishtiyaq Ahmed

 **License:** MIT

 **Note**: Formatted and documented the code using Artificial Intelligence

## **Prerequisites**

**1. Add your Google Gemini AI API Key as a secret in kaggle**

* In Kaggle > Add-ons > Secrets > + Add Secret > Label = GOOGLE_API_KEY and Value = Your_API_Key_Value

## **Generate a Web Application from a Topic and its Target Audience**

## Import the required modules

This cell imports necessary Python standard libraries (like `os`) and specialized modules from Kaggle secrets and Google ADK (AI Developer Kit). These imports include classes for AI agents, models (Google Gemini), runners, sessions, memory services, and tools such as agent tools and Google search. These foundational imports enable the use of AI language models and search integrations in the notebook.

In [15]:
# Import necessary modules for the application.
import os # Used for operating system functionalities, like environment variables.
from kaggle_secrets import UserSecretsClient # Used to securely retrieve API keys from Kaggle secrets.

# Import core components from the Google ADK (Agent Development Kit).
from google.adk.agents import (
    Agent, # Base class for defining individual AI agents.
    SequentialAgent, # Agent that orchestrates a sequence of other agents.
)
from google.adk.models.google_llm import Gemini # Imports the Gemini Large Language Model from Google.
from google.adk.runners import InMemoryRunner, Runner # Used to run agents; InMemoryRunner is for in-memory sessions.
from google.adk.sessions import InMemorySessionService, DatabaseSessionService # Manages conversation sessions.
from google.adk.memory import InMemoryMemoryService # Manages agent's memory for conversations.

# Import tools that agents can use.
from google.adk.tools.agent_tool import AgentTool # Base class for defining custom tools.
from google.adk.tools.google_search_tool import google_search # A pre-built tool for performing Google searches.
from google.genai import types # Import 'types' specifically for configuring HttpRetryOptions for the Gemini model.

# Hide additional warnings in the notebook
import warnings
warnings.filterwarnings("ignore")

print("Imported the required modules")

Imported the required modules


## Import the Gemini API Key from Kaggle secrets

This cell securely fetches the Gemini API key using Kaggle's `UserSecretsClient`. It assigns the key to an environment variable to be used for authentication with Google's Gemini API. It prints a success message if the key initializes correctly or an error message if there is an issue.

In [16]:
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")  # Retrieve Google Gemini API key securely
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY  # Set the API key as an environment variable for authentication
    print("Successfully initialized Google (Gemini) API Key!")  # Confirm successful initialization
except:
    # Handle any errors during secret retrieval or setting environment variable
    print("There was a problem initializing the API Key, Check whether the API key is correct and try again!")

Successfully initialized Google (Gemini) API Key!


## Initialize the retry configuration

Here, a retry policy is set up for HTTP requests to external APIs. It specifies:
- Maximum of 5 retry attempts on failure,
- The exponential base for backoff delay is 7,
- Initial delay of 1 second before retrying,
- HTTP status codes (429, 500, 503, 504) that will trigger a retry.

This helps the notebook handle temporary network or server errors gracefully.

In [17]:
# Configure HTTP retry options for API calls to handle transient errors gracefully
retry_config = types.HttpRetryOptions(
    attempts=5,                # Maximum number of retry attempts for failed requests
    exp_base=7,                # Exponential backoff multiplier for delay between retries
    initial_delay=1,           # Initial wait time in seconds before the first retry
    http_status_codes=[429, 500, 503, 504]  # HTTP status codes (rate limit, server issues) to trigger retry
)

# This setup helps automatically retry API calls that may fail due to rate limiting or temporary server errors,
# improving robustness of interactions with the Google Gemini API.

## **Meet the AI Agents Team!**

## Introduction to the AI Agents team workflow

This markdown cell outlines the workflow of sequential AI agents used in this project:
1. **Search Query Generator**: Creates optimized search queries on Google based on the given topic.
2. **Researcher**: Uses generated queries to gather accurate information.
3. **Domain Expert**: Provides expert opinions on the topic.
4. **Content Writer**: Drafts a content document.
5. **Content Moderator**: Polishes and filters content before final output.

This high-level overview explains the modular design of the content generation pipeline.

## Define the Prompt Enhancer Agent

Defines an AI agent named `promptenhancer` using the Gemini model "gemini-2.5-flash-lite" and the retry configuration. This agent improves user prompts to produce clearer, more educational prompts suitable for document creation. It uses the Google Search tool optionally to access general knowledge.

In [18]:
# Define the Prompt Enhancer Agent using the Gemini LLM with retry options
prompt_enhancer = Agent(
    name="prompt_enhancer",  # Unique name for the agent
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Use the Gemini model variant optimized for lighter, fast responses
        retry_options=retry_config  # Apply HTTP retry settings to handle transient API errors
    ),
    description="",  # Optional description (empty here, but can clarify agent's purpose)
    instruction="""
        You are an expert prompt engineer. 
        You are tasked with improving the given prompt to get better and accurate response. 
        This prompt would be used to generate an educational document for the given target audience.
    """,  # Clear instruction for the agent's role and task
    tools=[google_search],  # Include Google Search tool to enhance prompt improvement using search data
    output_key="enhanced_prompt"  # Output variable name for the enhanced prompt generated by this agent
)

## Define the Search Query Generator Agent

Defines `search_query_generator` agent using the same model and retry settings. This agent, acting as an SEO expert, generates a list of optimized search queries for Google based on the enhanced prompt.

In [19]:
# Define the Search Query Generator Agent using the Gemini model with retry options
search_query_generator = Agent(
    name="search_query_generator",  # Unique identifier for the agent
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Use the Gemini flash-lite model variant
        retry_options=retry_config  # Apply retry configuration for API robustness
    ),
    description="",  # Optional description; can be added for clarity
    instruction="""
        You are an SEO Expert, and a Google Search Query specialist.
        You are tasked with generating a list of optimal and relevant search queries
        to search on Google for the topic: {enhanced_prompt}
    """,  # Clear task defining the agent’s role in generating SEO-optimized search queries based on enhanced prompt
    tools=[],  # No additional tools are used by this agent in this setup
    output_key="optimal_queries_list"  # Output variable name to store generated list of search queries
)

## Define the Outliner Agent

The `outliner` agent is defined to create a structured outline of important subtopics for the document. It uses the list of optimized search queries and Google Search to assist in outlining.

In [20]:
# Define the Outliner Agent to create a structured outline for the topic
outliner = Agent(
    name="outliner",  # Unique name for the agent
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Use the lightweight Gemini model variant
        retry_options=retry_config  # Apply retry options for API reliability
    ),
    description="",  # Optional brief description for the agent's purpose
    instruction="""
        You need to make an outline of the important concepts to be included for the given topic. 
        You may use this list of optimal search queries {optimal_queries_list} and google_search tool.
    """,  # Instruction directing the agent to create a detailed topic outline using search queries and web search
    tools=[google_search],  # Enable Google Search tool to assist in outlining
    output_key="topic_outline"  # Output variable storing the generated topic outline
)

## Define the Researcher Agent

Defines the `researcher` agent configured to use search queries and web searches to gather detailed research findings for each subtopic in the outline.

In [21]:
# Define the Researcher Agent to perform detailed web research using search queries and outline
researcher = Agent(
    name="researcher",  # Agent identifier
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Gemini model variant for quick and efficient performance
        retry_options=retry_config  # Apply HTTP retry settings for robustness
    ),
    description="",  # Optional description for agent role
    instruction="""
        You are an expert researcher.
        You are tasked with using this list of optimal search queries: {optimal_queries_list} 
        and search them using google_search tool to gather relevant information about the given topic with citations from the internet.
        Follow the outline: {topic_outline}
        You need to generate content that is suitable for the target audience
    """,  # Instructions guiding the agent to research following the outline and focus on the target audience
    tools=[google_search],  # Enable Google Search tool for gathering real-time data
    output_key="research_findings"  # Output key that stores gathered research content with citations
)

## Define the Domain Expert Agent

Defines `domain_expert` agent that uses the collected research findings to provide expert domain insights and generate authoritative content relevant to the topic.

In [22]:
# Define the Domain Expert Agent to provide authoritative insights and detailed domain knowledge
domain_expert = Agent(
    name="domain_expert",  # Unique agent name
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Use Gemini light model for efficient generation
        retry_options=retry_config  # Apply retry config for resilient API usage
    ),
    description="",  # Optional role description
    instruction="""
        You are an expert in the given topic and its domain. 
        You are required to provide your expert opinions and generate in-depth and accurate information about the domain and its topic. 
        You can use google_search tool to search the internet along with the citations of the source where you got that information from on the internet.
    """,  # Clear instructions setting expectations for detail, authority, and citation
    tools=[google_search],  # Access to real-time Google Search for supporting data
    output_key="domain_expert_opinion"  # Output variable to store expert analysis and opinions
)

## Define the Content Writer Agent

Defines the `content_writer` agent that uses the research and expert opinions to compose the draft document content, aiming for clear, engaging writing tailored to the target audience.

In [23]:
# Define the Content Writer Agent to produce engaging, audience-tailored draft content
content_writer = Agent(
    name="content_writer",  # Agent identification
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Lightweight Gemini model for rapid generation
        retry_options=retry_config  # Retry settings to handle API request issues
    ),
    description="",  # Optional descriptive text about the agent role
    instruction="""
        You are an expert content writer who can write engaging content about a topic using the research: {research_findings} 
        and domain expert opinion: {domain_expert_opinion} for a given target audience. 
        You may use the optimal search queries: {optimal_queries_list} and google_search tool to get more information about the topic as required, 
        but you also need to mention the citations of the source where you got that information from on the internet.
    """,  # Detailed instruction emphasizing content quality, research integration, citation, and audience awareness
    tools=[google_search],  # Allows the agent to access more data if necessary via Google Search
    output_key="generated_RAW_content"  # Storage key for the raw draft content generated
)

## Define the Content Moderator Agent

Defines the `content_moderator` agent tasked with filtering, polishing, and converting the raw draft content into well-formed HTML code with CSS styling. Ensures ethical and relevant content with citations intact.

In [24]:
# Define the Content Moderator Agent to review, filter, and format the draft content into polished HTML
content_moderator = Agent(
    name="content_moderator",  # Unique name for the agent
    model=Gemini(
        model="gemini-2.5-flash-lite",  # Lightweight Gemini model for efficient processing
        retry_options=retry_config  # Retry configuration for API robustness
    ),
    description="",  # Optional description about the agent's role
    instruction="""
        You are an expert content moderator in the given topic and domain. 
        You may use the google_search tool to fill in any missing information. 
        You are tasked with filtering the content: {generated_RAW_content}, 
        make sure the content is relevant to the topic, accurate, 
        halal-friendly, and follows ethical considerations. 
        Then convert the content to HTML code and apply CSS for styling. 
        Do not mention your comments or feedback just give the code. 
        Do not include any graphics in the code. Use tags that only display text.
        Do not explicitly mention the target audience.
        Use CSS styles based on the target audience and topic.
        Give an AI generated content disclaimer to inform that the information may not be accurate at the bottom of the web page.
    """,  # Detailed instruction focusing on content relevance, accuracy, ethics, and formatting as styled HTML
    tools=[google_search],  # Allows the agent to search for missing info if needed
    output_key="filtered_content"  # Output key for the final moderated and formatted HTML content
)

In [25]:
# Define the root SequentialAgent to orchestrate the entire content generation pipeline
rootAgent = SequentialAgent(
    name="rootAgent",  # Unique name identifying this orchestrating agent
    sub_agents=[
        prompt_enhancer,         # Step 1: Refine user prompt for clarity and accuracy
        search_query_generator,  # Step 2: Generate optimized search queries based on enhanced prompt
        outliner,                # Step 3: Create structured topic outline from queries
        researcher,              # Step 4: Conduct detailed research based on outline and queries
        domain_expert,           # Step 5: Provide expert insights and authoritative content
        content_writer,          # Step 6: Draft the main content integrating research and expert opinion
        content_moderator        # Step 7: Filter, polish, and format content as HTML
    ],
)

## Setup Session Service and Runner

Initializes the `InMemorySessionService` to keep track of session data and an `InMemoryRunner` to execute the sequential pipeline of agents, passing along context, prompts, and outputs between each phase.

In [26]:
# Initialize an in-memory session service to manage conversation state during agent execution
session_service = InMemorySessionService()

# Create a Runner instance to execute the rootAgent sequential pipeline
# The runner manages the lifecycle and context of the agent workflow within the app "default"
runner = Runner(
    agent=rootAgent,            # The root SequentialAgent coordinating sub-agents
    app_name="default",         # Name of the application or workflow context
    session_service=session_service  # Session service to track conversation and state
)

## User Input for Topic and Audience

Prompts the user to enter a topic and a target audience for which the document will be generated.

In [27]:
# Prompt the user to enter the topic for which the document will be generated
print("When giving the topic, be specific and avoid acronyms for more accurate results!")
topic = input("Enter the topic you want to generate document for (eg., Artificial Intelligence): ")

# Prompt the user to enter the target audience for the document
target_audience = input("Enter the target audience for the document (eg., Graduates): ")

# Combine topic and target audience into a single prompt string for the AI agents
prompt = "Topic: " + topic + ", Target Audience: " + target_audience

When giving the topic, be specific and avoid acronyms for more accurate results!


Enter the topic you want to generate document for (eg., Artificial Intelligence):  Impact of Artificial Intelligence on Software Development
Enter the target audience for the document (eg., Graduates):  Software Engineers


## Running the sequential AI agents

Runs the entire agent pipeline in sequence with the user's input topic and target audience. Each agent's output feeds into the next, from prompt enhancement to final content moderation, producing the finished document.

In [28]:
# Run the rootAgent pipeline asynchronously using the prompt input,
# with debug enabled to track intermediate events and outputs
response = await runner.run_debug(prompt)

# Extract the last event from the response which contains the final output content
event = response[-1]

# Define a helper function to collect all text parts from the event content,
# concatenating them into a single continuous string
def get_all_text_from_event(event):
    texts = [part.text for part in event.content.parts]  # Extract text from each content part
    return "".join(texts)  # Combine all parts into one string

# Use the helper function to get the full text output from the final event
full_text = get_all_text_from_event(event)

# Print the complete generated document content (usually HTML from the moderator agent)
print(full_text)


 ### Created new session: debug_session_id

User > Topic: Impact of Artificial Intelligence on Software Development, Target Audience: Software Engineers
prompt_enhancer > The integration of Artificial Intelligence (AI) into software development is fundamentally transforming the role of software engineers, shifting their focus from manual coding and debugging to higher-level problem-solving, architectural planning, and strategic decision-making. This evolution is driven by AI's ability to automate routine tasks, enhance productivity, and improve the overall quality and efficiency of the software development lifecycle.

**Key Impacts of AI on Software Development:**

*   **Code Generation and Assistance:** AI-powered tools, such as GitHub Copilot and Amazon CodeWhisperer, can generate code snippets, entire functions, and even translate high-level concepts into executable code. This significantly reduces development time and minimizes human error. Developers are increasingly using AI to 

## Provide file link for viewing

Uses IPython's display functions to create a clickable link for viewing the saved HTML output file in the notebook environment.

In [29]:
from IPython.display import FileLink, display  # Import functions to create clickable file links in Jupyter

filename = "Web_Application.html"  # Define the output HTML file name

# Open the file in write mode with UTF-8 encoding and save the generated HTML content
with open(filename, "w", encoding="utf-8") as f:
    f.write(full_text[8:-3])  # Write the HTML content, slicing to remove unwanted leading/trailing characters if any

print(f"HTML saved to {filename}")  # Confirmation message

# Display a clickable link in the notebook that allows users to open or download the generated HTML file
display(FileLink(filename, result_html_prefix="Click here to view the generated web application file: "))
input("Press any enter to exit the program")

HTML saved to Web_Application.html


/kaggle/working/Web_Application.html

Press any enter to exit the program 


''

This approach modularizes the AI content generation process, combining prompt engineering, optimized search, expert input, and content refinement into an automatic document creator.